In [1]:
pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
import torch

# Check if GPU is available, otherwise fall back to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [4]:
# Load the GPT-2 tokenizer and model from Hugging Face
model_name = "gpt2"  # You can choose a different variant like 'gpt2-medium', 'gpt2-large', 'gpt2-xl'

# Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Move the model to the GPU (if available)
model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [5]:
def generate_text(prompt, max_length=100, temperature=1.0, top_k=50, top_p=0.95):
    # Encode the input prompt
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

    # Generate the text
    with torch.no_grad():  # Turn off gradients to save memory
        outputs = model.generate(
            input_ids=input_ids,
            max_length=max_length,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            num_return_sequences=1,  # Generate 1 sequence of text
            no_repeat_ngram_size=2,  # Prevent repeating phrases
            do_sample=True,  # Enable sampling to add randomness
            pad_token_id=tokenizer.eos_token_id  # Ensure padding doesn't break generation
        )

    # Decode the generated text back into readable form
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return generated_text


In [6]:
# Define the prompt text
prompt_text = "In the future, artificial intelligence will"

# Generate text based on the prompt using the GPU
generated_output = generate_text(prompt_text, max_length=150, temperature=0.7, top_k=50, top_p=0.9)

# Print the generated text
print("Generated Text:\n")
print(generated_output)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generated Text:

In the future, artificial intelligence will be able to provide the necessary skills for a truly intelligent society, and for the very same reason that the human mind is not an intelligent organism.

If we want to build a society that is truly sentient, then the basic task must be to create a system that can interact with our environment and our culture. That is to say, to use a computer to interact and to communicate with its environment, we need to be willing to do that. It is the same with a human being. In the world of artificial intelligences, the most basic requirement is that we have the ability to learn to understand and use our own human language. This is what we call "science."
 (For more on this,


In [7]:
# === Zero-Shot Prompt ===
def zero_shot_example():
    prompt = "Is it safe to travel to Mars? Answer with evidence and logic."
    result = generate_text(prompt, max_length=100, temperature=0.7)
    print("Zero-Shot Prompt Output:")
    print(result)
    print("\n" + "="*50 + "\n")

# === Few-Shot Prompt ===
def few_shot_example():
    prompt = """Translate English to French:
    English: Hello, how are you?
    French: Bonjour, comment ça va?

    English: What is your name?
    French:"""
    result = generate_text(prompt, max_length=100, temperature=0.7)
    print("Few-Shot Prompt Output:")
    print(result)
    print("\n" + "="*50 + "\n")

# === Chain-of-Thought Prompt ===
def chain_of_thought_example():
    prompt = """Let's solve this math problem step by step:
    What is 12 multiplied by 8?
    First, multiply 12 by 8, which gives:
    12 * 8 = 96. So, the answer is 96.
    Now, let's solve: What is 15 multiplied by 9?"""
    result = generate_text(prompt, max_length=100, temperature=0.7)
    print("Chain-of-Thought Prompt Output:")
    print(result)
    print("\n" + "="*50 + "\n")

# === React Prompt ===
def react_example():
    prompt = """If a customer asks: "Can you help me find a good smartphone?", react in a friendly and helpful manner. Provide the best suggestions and reasons.
    Customer: Can you help me find a good smartphone?
    Assistant:"""
    result = generate_text(prompt, max_length=100, temperature=0.7)
    print("React Prompt Output:")
    print(result)
    print("\n" + "="*50 + "\n")

# === Running the Examples ===

zero_shot_example()
few_shot_example()
chain_of_thought_example()
react_example()

Zero-Shot Prompt Output:
Is it safe to travel to Mars? Answer with evidence and logic.

If there are any obstacles to a successful landing, we need to make it clear to the world that we are not going to be able to land on the surface of Mars. We can't just walk on water and then walk back to Earth. That's not a safe and sustainable way to go. If we want to walk up to space, then we must first understand the risks. This is where NASA's Mars Exploration


Few-Shot Prompt Output:
Translate English to French:
    English: Hello, how are you?
    French: Bonjour, comment ça va?

    English: What is your name?
    French: Journé, answer è les mots, je vous

[Translation]



Chain-of-Thought Prompt Output:
Let's solve this math problem step by step:
    What is 12 multiplied by 8?
    First, multiply 12 by 8, which gives:
    12 * 8 = 96. So, the answer is 96.
    Now, let's solve: What is 15 multiplied by 9?  This gives the following: 
1 12 = 15
2 12 - 9 = 14.4
3 13 = 7.2
4


React Prompt O

In [10]:
!pip install langchain langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00


In [1]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

# Create a simple LLM wrapper using GPT-2
pipe = pipeline("text-generation", model="microsoft/phi-2", trust_remote_code=True)

llm = HuggingFacePipeline(pipeline=pipe)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
<ipython-input-1-7f45795bcbbb>:8: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)
<ipython-input-1-7f45795bcbbb>:28: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(final_prompt)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


=== GPT-2 Output ===

Review:

I bought this blender last week and it arrived in 3 days.


Read the product review and answer in 1 word How many days did it take to arrive? 


Answer: 3



In [ ]:
# Sample review
review = """
I bought this blender last week and it arrived in 3 days.
"""

# Prompt Template — simplified, no JSON
template = """
Review:
{review}

Read the product review and answer in one word How many days did it take to arrive?

"""

prompt = PromptTemplate(template=template, input_variables=["review"])
final_prompt = prompt.format(review=review)

# Run model
response = llm(final_prompt)
print("=== GPT-2 Output ===")
print(response)

In [8]:
delivery_days_schema = ResponseSchema(
    name="delivery_days",
    description="How many days did it take for the product to arrive?"
)

response_schemas = [delivery_days_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()


review = """
I bought this blender last week and it arrived in 3 days. It was a birthday gift for my sister.
"""

prompt_template = PromptTemplate(
    template="Analyze the following product review and extract structured information.\n\n{format_instructions}\n\nReview:\n{review}",
    input_variables=["review"],
    partial_variables={"format_instructions": format_instructions}
)

final_prompt = prompt_template.format(review=review)

print("\n=== final prompt ===")
print(final_prompt)

# 5. Run generation
response = llm(final_prompt)

# 6. Display raw + parsed output
print("=== Raw Response ===")
print(response)

print("\n=== Parsed Output ===")
try:
    structured_data = output_parser.parse(response)
    print(structured_data)
except Exception as e:
    print("Parsing failed:", e)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



=== final prompt ===
Analyze the following product review and extract structured information.

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"delivery_days": string  // How many days did it take for the product to arrive?
}
```

Review:

I bought this blender last week and it arrived in 3 days. It was a birthday gift for my sister.

=== Raw Response ===
Analyze the following product review and extract structured information.

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"delivery_days": string  // How many days did it take for the product to arrive?
}
```

Review:

I bought this blender last week and it arrived in 3 days. It was a birthday gift for my sister.
## INPUT

##OUTPUT
```json
{
	"delivery

=== Parsed Output ===
Parsing failed: Got invalid return object. Expected key `delive